# Truly suspicious events and false alarms detection
### 25.06.2023

### Import libraries

In [51]:
import pandas as pd
import os, os.path
import sqlite3
from sklearn.ensemble import IsolationForest
import py7zr

### Train and test data extraction to given path PATH

In [ ]:
with py7zr.SevenZipFile('cybersecurity_training.7z', mode='r') as z:
    z.extractall(path='PATH') 
with py7zr.SevenZipFile('cybersecurity_test.7z', mode='r') as z:
    z.extractall(path='PATH')

### Read data from given path PATH

In [7]:
Data = pd.read_csv('PATH/cybersecurity_training.csv', delimiter='|')
TestData = pd.read_csv('PATH/cybersecurity_test.csv', delimiter='|')

### Checking how many alerts were considered truly suspicious in train data using sql langauge

In [50]:
baza = os.path.join('przyklad.db')
if os.path.isfile(baza):
    os.remove(baza) 
conn = sqlite3.connect(baza) 

Data.to_sql("Data", conn)
Notified = pd.read_sql_query("""
SELECT COUNT(*) AS TotalNumber
FROM Data
GROUP BY notified
""", conn)
conn.close()

res = Notified['TotalNumber'][1]/(Notified['TotalNumber'][1] + Notified['TotalNumber'][0])
print(res)

0.057726938392472164


### Selecting columns for algorithm

In [16]:
SelectedColumns = Data[['correlatedcount','srcip_cd','dstip_cd','srcport_cd','dstport_cd','alerttype_cd','direction_cd','eventname_cd','severity_cd','reportingdevice_cd','devicetype_cd','devicevendor_cd','domain_cd','protocol_cd','username_cd','srcipcategory_cd','dstipcategory_cd','isiptrusted','untrustscore','flowscore','trustscore','enforcementscore','srcportcategory_dominate','thrcnt_month','thrcnt_week','thrcnt_day','p6','p9','p5m','p5w','p5d','p8m','p8w','p8d']]

### Model creating using IsolationForest for the problem

In [33]:
model=IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.05),max_features=1.0, random_state=42)
model.fit(SelectedColumns.values)

IsolationForest(contamination=0.05, n_estimators=50, random_state=42)

### Selecting columns for algorithm

In [39]:
TestSelectedColumns = TestData[['correlatedcount','srcip_cd','dstip_cd','srcport_cd','dstport_cd','alerttype_cd','direction_cd','eventname_cd','severity_cd','reportingdevice_cd','devicetype_cd','devicevendor_cd','domain_cd','protocol_cd','username_cd','srcipcategory_cd','dstipcategory_cd','isiptrusted','untrustscore','flowscore','trustscore','enforcementscore','srcportcategory_dominate','thrcnt_month','thrcnt_week','thrcnt_day','p6','p9','p5m','p5w','p5d','p8m','p8w','p8d']]

### Results for test data

In [55]:
Testresult = pd.DataFrame()
Testresult['scores']=model.decision_function(TestSelectedColumns.values)
Testresult['anomaly']=model.predict(TestSelectedColumns.values)

### Saving the score column to a text file

In [41]:
Columndata = Testresult['scores']
with open('results.txt', 'w') as file:
    for item in Columndata:
        file.write(str(item) + '\n')